<a href="https://colab.research.google.com/github/3B032070/2025_ML_hws/blob/main/hw5_1_colab_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# hw5-1: IMDb 情意分析訓練模型

本 Notebook 符合作業要求：
- IMDb 資料下載與預處理
- 建立 Keras 模型
- 顯示各層參數與 shape
- 訓練過程分析
- 儲存模型與 tokenizer

In [6]:
# 下載與解壓縮 IMDb 資料
!mkdir -p data
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz -P data
!tar -xzf data/aclImdb_v1.tar.gz -C data

--2025-05-22 06:32:24--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘data/aclImdb_v1.tar.gz.1’

aclImdb_v1.tar.gz.1 100%[===================>]  80.23M  28.1MB/s    in 2.9s    

2025-05-22 06:32:27 (28.1 MB/s) - ‘data/aclImdb_v1.tar.gz.1’ saved [84125825/84125825]



In [7]:
# 顯示資料夾結構
import os
for root, dirs, files in os.walk("data/aclImdb"):
    level = root.replace("data/aclImdb", "").count(os.sep)
    indent = " " * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    for f in files[:2]:
        print(f"{indent}    {f}")

aclImdb/
    imdbEr.txt
    imdb.vocab
    test/
        labeledBow.feat
        urls_neg.txt
        pos/
            696_10.txt
            1402_8.txt
        neg/
            8110_2.txt
            4851_4.txt
    train/
        labeledBow.feat
        urls_neg.txt
        pos/
            10614_7.txt
            6068_9.txt
        unsup/
            7163_0.txt
            14133_0.txt
        neg/
            467_1.txt
            9470_1.txt


In [8]:
# 資料讀取與清洗
import numpy as np
import re
from os import listdir

def rm_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def read_files(filetype):
    path = f"data/aclImdb/{filetype}/"
    texts, labels = [], []
    for sentiment in ['pos', 'neg']:
        dir_path = path + sentiment
        for file in listdir(dir_path):
            with open(f"{dir_path}/{file}", encoding="utf-8") as f:
                texts.append(rm_tags(f.read()))
                labels.append(1 if sentiment == 'pos' else 0)
    return texts, labels

train_text, y_train = read_files("train")
test_text, y_test = read_files("test")

In [5]:
# Tokenizer 處理
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 5000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_text)

x_train = tokenizer.texts_to_sequences(train_text)
x_test = tokenizer.texts_to_sequences(test_text)

maxlen = 300
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [9]:
# 建立模型
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=32, input_length=maxlen))  # 5000*32=160000
model.add(Flatten())  # (None, 9600)
model.add(Dense(64, activation='relu'))  # 9600*64+64
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # 64*1+1

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
# 編譯與訓練
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.2
)

ValueError: Argument `validation_split` is only supported for tensors or NumPy arrays.Found incompatible type in the input: [<class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <clas

In [ ]:
# 繪製訓練圖表
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')
plt.show()

plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy')
plt.show()

In [ ]:
# 評估模型
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

In [ ]:
# 儲存模型與 tokenizer
model.save("imdb_model.h5")

import pickle
with open("tokenizer.pickle", "wb") as f:
    pickle.dump(tokenizer, f)